In [122]:
import os
import pandas as pd
import json
import pprint
import re
import nbformat

In [134]:
# Define the path
#path = os.path.join("GitHub", "DataProcessing")

# Print the path
#print(path)

In [137]:
#pwd

In [139]:
#td = pd.read_json("..\DataProcessing\TestData\message.json", orient='index')

In [143]:
# Read the JSON file
with open("../DataProcessing/TestData/message.json", "r") as file:
    data = json.load(file)

In [142]:
# Access the 'participants' array within 'metadata'
participants_data = data['metadata']['participants']
participants_data

KeyError: 'metadata'

In [128]:
puuidPair = data['info']['participants']
puuidPair

[{'participantId': 1,
  'puuid': 'NLZhdGCfh-EgdvaODKW2IlCFzJBMtE4Dr9SFgKNpE1UC6GDNH2u2TQmGZoqSLXcVjo-erCaRQVV0cQ'},
 {'participantId': 2,
  'puuid': 'vfktkyVovXtnWWgFEDKxdtfJFVHSV3tAJeoT_UjbZB8TlE8tyVYLyC23nudviaAwk0c6AkFGGTYJhQ'},
 {'participantId': 3,
  'puuid': 'O9jmQXkdmuplQq2s4YGCd-x2VnJetyQ7ssrBczKK-xwT0PYaVpMyOWIHX_iIIznvHmk0M5_uE_CRzw'},
 {'participantId': 4,
  'puuid': 'HHVQBTfTWCQzCredc0AtS4BotTQqhQhK_7T4241WyfhlVNI1Oy7SHDjpPOg4T0KOuHu7eIS8gsRXsA'},
 {'participantId': 5,
  'puuid': '-e-lfG5zpkNcr96I_Q_5ZdDefVsot-QfGnTH2QSHv-cMyP1_u8NUf-hImv_nQKwCUIq05DF8yFAIEw'},
 {'participantId': 6,
  'puuid': 'tTu5MniQjpI3G_eEm31jSm8ZhwCYJjRUQgQu0ynIs05DxKEkiRMdpOBq_3axazOPR4eP-M_gYcsHxg'},
 {'participantId': 7,
  'puuid': '8s-H0x0EL9H84mDtglkoIYtyxASxccITkQaICj8RKUore3nABvXCZg_apnRSXMwLJYd2nCDUkJVk_A'},
 {'participantId': 8,
  'puuid': 'JBQKhP0EHYxtzSCrglHYSWq8of63qVowOOQ9NyMm6Rfv_Ups7-qrn11AgSY9-RoKro32bTDMsAqF3A'},
 {'participantId': 9,
  'puuid': 'y_0f-NLP6I974-LIJDU7WDu4qdHkqjVDaOUODj

In [129]:
# Get every timestamp within a game paired with corresponding minute
for counter, frame in enumerate(data['info']['frames']):
    timestamp = frame['timestamp']
    print(f"{counter + 1} Timestamp = {timestamp}")

1 Timestamp = 0
2 Timestamp = 60016
3 Timestamp = 120021
4 Timestamp = 180035
5 Timestamp = 240035
6 Timestamp = 300066
7 Timestamp = 360069
8 Timestamp = 420084
9 Timestamp = 480090
10 Timestamp = 540109
11 Timestamp = 600126
12 Timestamp = 660130
13 Timestamp = 720141
14 Timestamp = 780169
15 Timestamp = 840170
16 Timestamp = 900173
17 Timestamp = 960186
18 Timestamp = 1020272
19 Timestamp = 1080286
20 Timestamp = 1140318
21 Timestamp = 1200328
22 Timestamp = 1260354
23 Timestamp = 1320366
24 Timestamp = 1380386
25 Timestamp = 1440411
26 Timestamp = 1500414
27 Timestamp = 1560433
28 Timestamp = 1620447
29 Timestamp = 1680454
30 Timestamp = 1740480
31 Timestamp = 1800496
32 Timestamp = 1860523
33 Timestamp = 1892560


In [130]:
# Pull specific data for specified participant id at specific minute
def get_participant_data_at_minute(data, participant_id, target_minute, display_nozero=False):
    timestamp_data = []

    def display_nonzero_params(participant_frame):
        # Iterate through the participant frame's items
        for key, value in participant_frame.items():
            if isinstance(value, dict):
                print(f"{key}:")
                # If the value is a nested dictionary, iterate through its items
                display_nonzero_params(value)
            elif isinstance(value, (int, float)) and (not display_nozero or value != 0):
                # Display the parameter if it's a numeric value and not zero
                print(f"{key}: {value}")

    # Check if there are frames in the data
    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            timestamp = frame['timestamp']
            current_minute = round(timestamp / 60000)  # Convert milliseconds to minutes and round to the nearest minute

            if current_minute == target_minute:
                participant_frames = frame.get('participantFrames', {})
                if participant_id in participant_frames:
                    participant_data = participant_frames[participant_id]
                    timestamp_data.append([timestamp, participant_data])

                    # Display nonzero parameters if specified
                    if display_nozero:
                        print(f"Timestamp at minute {target_minute}: {timestamp}")
                        display_nonzero_params(participant_data)
                        print("\n---\n")

    return timestamp_data

desired_participant_id = "2"
desired_minute = 5 

# Call the function with display_nozero set to True
result = get_participant_data_at_minute(data, desired_participant_id, desired_minute, display_nozero=True)


Timestamp at minute 5: 300066
championStats:
armor: 39
attackDamage: 72
attackSpeed: 122
health: 809
healthMax: 853
healthRegen: 17
magicResist: 36
movementSpeed: 365
currentGold: 1308
damageStats:
magicDamageDone: 3396
magicDamageDoneToChampions: 285
magicDamageTaken: 559
physicalDamageDone: 10717
physicalDamageDoneToChampions: 763
physicalDamageTaken: 2407
totalDamageDone: 19556
totalDamageDoneToChampions: 1079
totalDamageTaken: 2971
trueDamageDone: 5442
trueDamageDoneToChampions: 30
trueDamageTaken: 4
jungleMinionsKilled: 28
level: 4
minionsKilled: 4
participantId: 2
position:
x: 394
y: 461
timeEnemySpentControlled: 130346
totalGold: 2058
xp: 1611

---



In [131]:
# [NOTFINISHED] Get killed within specified timestamps 
def getParticipantIdByKill(data, start_timestamp, end_timestamp):
    for frame in data['info']['frames']:
        timestamp = frame['timestamp']
        
        for event in frame.get('events', []):
            if event['type'] == 'CHAMPION_KILL':
                kill_timestamp = event['timestamp']
                
                if start_timestamp <= kill_timestamp <= end_timestamp:
                    print(f"Timestamp of CHAMPION_KILL event: {kill_timestamp}")
                    print("Victim Damage Dealt:")
                    
                    # Assuming 'victimDamageDealt' is a list in the 'CHAMPION_KILL' event
                    for damage_info in event.get('victimDamageDealt', []):
                        print(f"  Name: {damage_info['name']}")

# Example usage: Specify the start and end timestamps
start_timestamp = 240035  # Replace with your desired start timestamp
end_timestamp = 540109    # Replace with your desired end timestamp

# Call the function with the specified time frame
getParticipantIdByKill(data, start_timestamp, end_timestamp)


Timestamp of CHAMPION_KILL event: 255814
Victim Damage Dealt:
  Name: Kayn
  Name: Kayn
Timestamp of CHAMPION_KILL event: 260683
Victim Damage Dealt:
  Name: Sion
  Name: Sion
  Name: Sion
  Name: Sion
Timestamp of CHAMPION_KILL event: 303299
Victim Damage Dealt:
  Name: Nautilus
  Name: Nautilus
  Name: Nautilus
  Name: Nautilus
  Name: Nautilus
  Name: Nautilus
  Name: Nautilus
Timestamp of CHAMPION_KILL event: 352366
Victim Damage Dealt:
  Name: Kayn
Timestamp of CHAMPION_KILL event: 389728
Victim Damage Dealt:
  Name: Sion
  Name: Sion
  Name: Sion
  Name: Sion
  Name: Sion
  Name: Sion
Timestamp of CHAMPION_KILL event: 413939
Victim Damage Dealt:
  Name: Veigar
  Name: Veigar
  Name: Veigar
Timestamp of CHAMPION_KILL event: 439097
Victim Damage Dealt:
  Name: Volibear
  Name: Volibear
  Name: Volibear
  Name: Volibear
Timestamp of CHAMPION_KILL event: 494299
Victim Damage Dealt:
  Name: Brand
  Name: Brand
  Name: Brand
  Name: Brand
  Name: Brand
  Name: Brand
  Name: Brand
Times

In [132]:
# Get timestamp of every kill in the game
def get_timestamp_of_champion_kill(data):
    for frame in data['info']['frames']:
        for event in frame.get('events', []):
            if event['type'] == 'CHAMPION_KILL':
                timestamp = event['timestamp']
                print(f"Timestamp of CHAMPION_KILL: {timestamp}")
        
                
# Call the function
get_timestamp_of_champion_kill(data)


Timestamp of CHAMPION_KILL: 134226
Timestamp of CHAMPION_KILL: 171327
Timestamp of CHAMPION_KILL: 187450
Timestamp of CHAMPION_KILL: 192190
Timestamp of CHAMPION_KILL: 209499
Timestamp of CHAMPION_KILL: 255814
Timestamp of CHAMPION_KILL: 260683
Timestamp of CHAMPION_KILL: 303299
Timestamp of CHAMPION_KILL: 352366
Timestamp of CHAMPION_KILL: 389728
Timestamp of CHAMPION_KILL: 413939
Timestamp of CHAMPION_KILL: 439097
Timestamp of CHAMPION_KILL: 494299
Timestamp of CHAMPION_KILL: 529933
Timestamp of CHAMPION_KILL: 568150
Timestamp of CHAMPION_KILL: 622319
Timestamp of CHAMPION_KILL: 627784
Timestamp of CHAMPION_KILL: 636359
Timestamp of CHAMPION_KILL: 650204
Timestamp of CHAMPION_KILL: 669689
Timestamp of CHAMPION_KILL: 715143
Timestamp of CHAMPION_KILL: 732110
Timestamp of CHAMPION_KILL: 740346
Timestamp of CHAMPION_KILL: 750514
Timestamp of CHAMPION_KILL: 775910
Timestamp of CHAMPION_KILL: 791129
Timestamp of CHAMPION_KILL: 866378
Timestamp of CHAMPION_KILL: 892824
Timestamp of CHAMPIO

In [133]:
# [NOTFINSHED] Print out all champion kill information at specific kill timestamp
def get_event_of_champion_kill_at_timestamp(data, target_timestamp):
    for frame in data['info']['frames']:
        for event in frame.get('events', []):
            if event['type'] == 'CHAMPION_KILL' and event['timestamp'] == target_timestamp:
                return event

# Example: Get CHAMPION_KILL event data at timestamp 134226
target_timestamp = 134226
champion_kill_event = get_event_of_champion_kill_at_timestamp(data, target_timestamp)

# Print or use the champion_kill_event data as needed
if champion_kill_event:
    print(f"CHAMPION_KILL event data:")
    pprint.pprint(champion_kill_event)
else:
    print(f"No CHAMPION_KILL event found at timestamp {target_timestamp}.")


CHAMPION_KILL event data:
{'bounty': 400,
 'killStreakLength': 0,
 'killerId': 1,
 'position': {'x': 3958, 'y': 13447},
 'shutdownBounty': 0,
 'timestamp': 134226,
 'type': 'CHAMPION_KILL',
 'victimDamageDealt': [{'basic': False,
                        'magicDamage': 0,
                        'name': 'Sion',
                        'participantId': 1,
                        'physicalDamage': 233,
                        'spellName': 'sionq',
                        'spellSlot': 0,
                        'trueDamage': 0,
                        'type': 'OTHER'},
                       {'basic': True,
                        'magicDamage': 0,
                        'name': 'Sion',
                        'participantId': 1,
                        'physicalDamage': 105,
                        'spellName': 'sionbasicattack3',
                        'spellSlot': 66,
                        'trueDamage': 0,
                        'type': 'OTHER'},
                       {'basic': Fa